In [23]:
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt

In [3]:
"""
config_map = {
    "model": config.ModelConfig(),
    "diffuser": config.DiffusionConfig(),
    "preprocess": config.PreprocessConfig(),
    "seq_diffuser": config.SeqDiffusionConfig(),
    
}
IGNORE_KEYS = ["d_time_emb", "d_time_emb_proj", "use_motif_timestep", "prob_self_cond", "str_self_cond"]
def compare_recurse(dict1, dict2, prefix=""):
    for k, v in dict1.items():
        if k in IGNORE_KEYS:
            continue
        if k not in dict2:
            print(f"Missing {prefix} {k}={v} in dict2")
        elif isinstance(v, dict):
            compare_recurse(v, dict2[k], prefix=f"{prefix}_{k}")
        elif v != dict2[k]:
            print(f"Mismatch {prefix} {k}={v} to {dict2[k]} (default)")
    return      

conf_dict = ckpt["config_dict"]
key_diff = set(conf_dict.keys()).difference(set(config_map.keys()))
if any(key_diff):
    print("Missing def for ", key_diff)
    
for k, dataclass_conf in config_map.items():
    if k in conf_dict:
        compare_recurse(conf_dict[k], asdict(dataclass_conf), prefix=k)
"""
print("ignore")

ignore


In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import py3Dmol
import torch
import numpy as np

from proteome import protein
from proteome.models.design.rfdiffusion import config
from proteome.models.design.rfdiffusion.modeling import RFDiffusionForUnconditionalStructureDesign, RFDiffusionForScaffoldGuidedBinderDesign
from proteome.models.design.rfdiffusion.samplers import SelfConditioning, ScaffoldedSampler
from proteome.models.design.rfdiffusion.util import writepdb

In [9]:
pdb = "1qys.pdb"
with open(pdb, mode="r") as f:
    pdb_str = f.read()
    
input_structure = protein.from_pdb_string(pdb_str, atom14_format=True, parse_hetatom=True)

In [10]:
with open('insulin_target.pdb', mode="r") as f:
    target_pdb_str = f.read()

with open('5L33.pdb', mode="r") as f:
    scaffold_pdb_str = f.read()

In [11]:
designer = RFDiffusionForScaffoldGuidedBinderDesign()
sampler = ScaffoldedSampler(
    model=designer.model,
    diffuser_conf=config.BaseConfig.diffuser,
    preprocess_conf=config.BaseConfig.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(),
    symmetry_conf=config.SymmetryParams(symmetry=None),
    ppi_conf=config.PPIParams(hotspot_res=["A59", "A83", "A91"]),
    denoiser_conf=config.DenoiserParams(noise_scale_ca=0, noise_scale_frame=0),
    scaffold_conf=config.ScaffoldGuidedParams(scaffoldguided=True, target_pdb_str=target_pdb_str, scaffold_pdb_str_list=[scaffold_pdb_str], target_ss=True, target_adj=True),
)

Downloading: "http://files.ipd.uw.edu/pub/RFdiffusion/60f09a193fb5e5ccdc4980417708dbab/Complex_Fold_base_ckpt.pt" to /home/conradry71/.cache/torch/hub/checkpoints/complex_fold_base.pt
100%|████████████████████████████████████████████████████████| 461M/461M [00:07<00:00, 68.0MB/s]


Successful diffuser __init__
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file . Best match rsd_type:  GLU:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 2 in file . Best match rsd_type:  VAL
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 3 in file . Best match rsd_type:  CYS
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 4 in file . Best match rsd_type:  PRO
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 6 in file . Best match rsd_type:  MET
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 7 in file . Best match rsd_type:  ASP
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 8 in file . Best match rsd_type:  ILE
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 5 atoms at position 9 in fil

In [12]:
x_init, seq_init = sampler.sample_init()

core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 4 atoms at position 1 in file . Best match rsd_type:  HIS:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 2 in file . Best match rsd_type:  MET
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 3 in file . Best match rsd_type:  PRO
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 4 in file . Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 5 in file . Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 6 in file . Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 7 in file . Best match rsd_type:  LYS
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 5 atoms at position 10 in file . Best match rsd_type:  AR

In [13]:
denoised_xyz_stack = []
px0_xyz_stack = []
seq_stack = []
plddt_stack = []

x_t = torch.clone(x_init)
seq_t = torch.clone(seq_init)
# Loop over number of reverse diffusion time steps.
for t in range(int(sampler.t_step_input), sampler.inf_conf.final_step - 1, -1):
    px0, x_t, seq_t, plddt = sampler.sample_step(
        t=t, x_t=x_t, seq_init=seq_t, final_step=sampler.inf_conf.final_step
    )
    px0_xyz_stack.append(px0)
    denoised_xyz_stack.append(x_t)
    seq_stack.append(seq_t)
    plddt_stack.append(plddt[0])

AssertionError: The checkpoint you're using hasn't been trained with sec-struc/block adjacency features

In [154]:
designer = RFDiffusionForUnconditionalStructureDesign()
sampler = SelfConditioning(
    model=designer.model,
    diffuser_conf=config.BaseConfig.diffuser,
    preprocess_conf=config.BaseConfig.preprocess,
    inference_conf=config.InferenceConfig(input_structure=input_structure, num_designs=1),
    contigmap_conf=config.ContigMap(contigs=["64-64"]),
    symmetry_conf=config.SymmetryParams(symmetry=None),
)

Successful diffuser __init__


In [155]:
x_init, seq_init = sampler.sample_init()

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [156]:
denoised_xyz_stack = []
px0_xyz_stack = []
seq_stack = []
plddt_stack = []

x_t = torch.clone(x_init)
seq_t = torch.clone(seq_init)
# Loop over number of reverse diffusion time steps.
for t in range(int(sampler.t_step_input), sampler.inf_conf.final_step - 1, -1):
    px0, x_t, seq_t, plddt = sampler.sample_step(
        t=t, x_t=x_t, seq_init=seq_t, final_step=sampler.inf_conf.final_step
    )
    px0_xyz_stack.append(px0)
    denoised_xyz_stack.append(x_t)
    seq_stack.append(seq_t)
    plddt_stack.append(plddt[0])

In [157]:
# Flip order for better visualization in pymol
denoised_xyz_stack = torch.stack(denoised_xyz_stack)
denoised_xyz_stack = torch.flip(
    denoised_xyz_stack,
    [
        0,
    ],
)
px0_xyz_stack = torch.stack(px0_xyz_stack)
px0_xyz_stack = torch.flip(
    px0_xyz_stack,
    [
        0,
    ],
)

# For logging -- don't flip
plddt_stack = torch.stack(plddt_stack)

In [158]:
# Save outputs
final_seq = seq_stack[-1]

# Output glycines, except for motif region
final_seq = torch.where(
    torch.argmax(seq_init, dim=-1) == 21, 7, torch.argmax(seq_init, dim=-1)
)  # 7 is glycine

bfacts = torch.ones_like(final_seq.squeeze())
# make bfact=0 for diffused coordinates
bfacts[torch.where(torch.argmax(seq_init, dim=-1) == 21, True, False)] = 0

In [159]:
out = f"test.pdb"
# Now don't output sidechains
writepdb(
    out,
    denoised_xyz_stack[0, :, :4],
    final_seq,
    sampler.binderlen,
    chain_idx=sampler.chain_idx,
    bfacts=bfacts,
)

In [166]:
with open("test.pdb", mode="r") as f:
    designed_pdb = f.read()

In [165]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(designed_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Folding

In [139]:
from proteome import protein
from proteome.models.design.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

In [140]:
sequence_designer = ProteinMPNNForSequenceDesign("vanilla_model-20", random_seed=37)
folder = OmegaFoldForFolding()

In [143]:
target_structure = protein.from_pdb_string(designed_pdb, chain_id="A")
target_structure = protein.to_backbone_only_protein(target_structure)

In [144]:
sequence, score = sequence_designer.design_sequence(target_structure)
print(f"Sequence: {sequence} with global_score {score}")

Sequence: AALAAAAAALAAAHAAAHAAAAAALAALAAADPAAAAALGAAAAAAHAAAGAAHAAAAALAAAA with global_score 0.8552732467651367


In [151]:
predicted_protein, confidence = folder.fold(sequence)
folded_pdb = protein.to_pdb(predicted_protein)

In [152]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(folded_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol